In [1]:
import os
import urllib.request as request
import zipfile
import shutil
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [2]:
%pwd

'd:\\Proyectos\\MLOps\\Chicken_Disease_Classification_MLOps_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Proyectos\\MLOps\\Chicken_Disease_Classification_MLOps_Project'

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config        


In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists and the size is {get_size}.")
    
    
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Use shutil to extract the file
        try:
            shutil.unpack_archive(self.config.local_data_file, unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")
        except shutil.ReadError:
            logger.error(f"Failed to extract {self.config.local_data_file}. It might not be a valid archive.")

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-18 13:40:47,559: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-18 13:40:47,563: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-18 13:40:47,565: INFO: common: created directory at: artifacts]
[2024-10-18 13:40:47,566: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-18 13:40:47,568: INFO: 3030581333: artifacts/data_ingestion/data.zip already exists and the size is <function get_size at 0x000001A317EAA550>.]
[2024-10-18 13:40:55,285: INFO: 3030581333: Extracted artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
